<h1 style="color:#D30982;text-align:center;vertical-align:middle;">Introduction to Variational Quantum Algorithms (VQA)
    <br></h1>

<h1 style="color:#D30982;">Overview</h1>

- What are VQAs?
- VQA Examples:
    - QAOA: Quantum Approximate Optimization Algorithm
    - VQE: Variational Quantum Eigensolver
    - QNNs: Quantum Neural Networks
    - VQLS: The Variational Quantum Linear Solver

<h1 style="color:#510981;">What are VQAs?</h1>

Variational Quantum Algorithms are posed as a leading method for getting near-term quantum advantage with NISQ devices. They combine quantum and classical compute resources such that a classical optimizer tunes the parameters of some quantum circuit in order to achieve a desired result. 

These algorithms involve preparing a quantum state, called a trial state or ansatz, that approximates the solution to the optimization problem, and then measures the expectation value of an observable, which represents the objective function to be optimized. The goal is to find the trial state that minimizes the expectation value of the observable, which corresponds to finding the optimal solution to the optimization problem.

Quantum variational algorithms are an active area of research and development in the field of quantum computing, with the potential to provide significant advances in solving optimization problems and other applications.

<h2 style="color:#D10283;">The Common Steps in all VQAs</h2>

All VQAs have the same steps but different methodologies. The steps they share are:

- Prepare a parameterized circuit: This means a circuit where some of the gates have parameters that allow us to fine-tune their effect (i.e. the angle of a CX rotation).

- Do some measurements: This could include multiple measurements in different basis.


- Calculate the cost of the solution given by your last run of the algorithm. Classically optimize the parameters of your circuit so that you minimize / maximize your cost function. This second point is also the reason why very often this algorithms are also called “hybrid”. The optimization part happens on a classical computer.


<h1 style="color:#D30982;">Examples of VQAs</h1>

There is a lot of research being done on VQAs. Some of the most common classes of VQAs are:

- VQE: The Variational Quantum Eigensolver is used to approximate the lowest energy level of a given Hamiltonian.
- QAOA: The Quantum Approximate Optimization Algorithm is mostly used for Combinatorial Optimization problems.
- QNNs: Quantum Neural Networks. Quantum analogues or generalizations of classical neural nets.
- VQLS: The Variational Quantum Linear Solver is used to solve systems of linear equations.

<h1 style="color:#510981;">VQE: The Variational Quantum Eigensolver </h1>

The Variational Quantum Eigensolver (VQE) is a quantum algorithm for finding the ground state energy of a quantum system using a quantum computer. It is a hybrid algorithm that combines classical and quantum computations.

<h3 style="color:#D10283;">The VQE algorithm involves three main steps</h3>


- preparing the parametrized quantum state, called the ansatz, that approximates a guess for the Hamiltonian's ground-state 
- measuring the energy of the ansatz using a quantum computer.
- updating the parameters of the circuit such that they move toward a minimum of the cost-function (ground-state)

**a parametrized circuit is used to prepare the ansatz. A parametrized circuit is a quantum circuit with adjustable parameters that can be tuned to minimize the energy of the ansatz. **

The energy of the ansatz is measured using a quantum computer. The energy is calculated by measuring the expectation value of the Hamiltonian, which is a Hermitian operator that represents the energy of the system. The measurement is repeated several times to obtain an estimate of the energy with a certain level of accuracy.

The VQE algorithm can be iterated by adjusting the parameters of the variational circuit based on the measurement results, until a minimum energy is obtained. The minimum energy corresponds to the approximate ground state energy of the system, and the corresponding state created by the parametrized quantum circuit is an approximation to the ground state wavefunction.

<h3 style="color:#D10283;">VQE Applications</h3>


- Quantum chemistry is one of the largest focuses for applying VQE. With quantum hardware that can achieve higher qubit counts and deeper circuits, it should be possible to conduct calculations relevant to developing pharmaceuticals, understanding protein dynamics, and even developing new quantum-relevant materials.


<h3 style="color:#D10283;">Example VQE Code</h3>


In [ ]:
# Define Hamiltonian corresponding to H2

from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

print(f"Number of qubits: {H2_op.num_qubits}")

from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.opflow import PauliSumOp

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=PauliSumOp(H2_op))
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

In [ ]:
# define ansatz and optimizer
from qiskit.circuit.library import TwoLocal
from qiskit.algorithms.optimizers import SPSA
# define Aer Estimator for noiseless statevector simulation
from qiskit.utils import algorithm_globals
from qiskit_aer.primitives import Estimator as AerEstimator
import pylab


def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    
iterations = 125
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
spsa = SPSA(maxiter=iterations)

# define callback
# note: Re-run this cell to restart lists before training
counts = []
values = []
seed = 170
algorithm_globals.random_seed = seed

noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)

# instantiate and run VQE
from qiskit.algorithms.minimum_eigensolvers import VQE

vqe = VQE(
    noiseless_estimator, ansatz, optimizer=spsa, callback=store_intermediate_result
)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)

print(f"VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}")
print(
    f"Delta from reference energy value is {(result.eigenvalue.real - ref_value):.5f}"
)

pylab.rcParams["figure.figsize"] = (12, 4)
pylab.plot(counts, values)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Convergence with no noise")

<h1 style="color:#D30982;">QAOA: The Quantum Approximate Optimization Algorithm</h1>

The Quantum Approximate Optimization Algorithm (QAOA) is a quantum algorithm for solving combinatorial optimization problems. It is a variational algorithm that uses a sequence of quantum gates to approximate the solution to an optimization problem.

<h3 style="color:#D10283;">The QAOA algorithm involves two main steps</h3>


- preparing a parametrized quantum state, called the ansatz, that encodes the solution to the optimization problem
- measuring the expectation value of an objective function.
- updating the parameters of the circuit such that they move toward a minimum of the cost-function (solution to the optimization problem)

In the first step, the ansatz is prepared using a sequence of quantum gates that generate a superposition of all possible solutions to the optimization problem. The ansatz is constructed using a parameterized circuit that can be optimized to improve the approximation to the optimal solution.

The objective function is measured using a quantum computer. The objective function is a classical function that maps each possible solution to a numerical value, which represents the objective to be optimized.

The QAOA algorithm can be iterated by adjusting the parameters of the ansatz based on the measurement results, until a satisfactory approximation to the optimal solution is obtained. The optimal solution corresponds to the solution that maximizes/minimizes the objective function, which represents the solution to the optimization problem.

<h3 style="color:#D10283;">QAOA Applications</h3>


- logistics
- finance
- machine learning. 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def compute_expectation(counts, G):
    
    avg = 0
    sum_count = 0
    for bitstring, count in counts.items():
        
        obj = maxcut_obj(bitstring[::-1], G)
        avg += obj * count
        sum_count += count
        
    return avg/sum_count


# We will also bring the different circuit components that
# build the qaoa circuit under a single function
def create_qaoa_circ(G, theta):
    
    nqubits = len(G.nodes())
    p = len(theta)//2  # number of alternating unitaries
    qc = QuantumCircuit(nqubits)
    
    beta = theta[:p]
    gamma = theta[p:]
    
    # initial_state
    for i in range(0, nqubits):
        qc.h(i)
    
    for irep in range(0, p):
        
        # problem unitary
        for pair in list(G.edges()):
            qc.rzz(2 * gamma[irep], pair[0], pair[1])

        # mixer unitary
        for i in range(0, nqubits):
            qc.rx(2 * beta[irep], i)
            
    qc.measure_all()
        
    return qc

# Finally we write a function that executes the circuit on the chosen backend
def get_expectation(G, shots=512):

    backend = Aer.get_backend('qasm_simulator')
    backend.shots = shots
    
    def execute_circ(theta):
        
        qc = create_qaoa_circ(G, theta)
        counts = backend.run(qc, seed_simulator=10, 
                             nshots=512).result().get_counts()
        
        return compute_expectation(counts, G)
    
    return execute_circ

<h1 style="color:#510981;">QNNs: Quantum Neural Networks</h1>

A neural network is ultimately just an elaborate function that is built by composing smaller building blocks called neurons. A neuron is typically a simple, easy-to-compute, and nonlinear function that maps one or more inputs to a single real number. The single output of a neuron is typically copied and fed as input into other neurons. 

<img src="../assets/Neural_network_example.svg" alt="isolated" width="200"/>

Neurons are respresented as nodes in a graph and we draw directed edges between nodes to indicate how the output of one neuron will be used as input to another. Each edge in our graph is often associated with a scalar-value called a weight

Quantum neural networks are quantum algorithms that are designed to simulate the behavior of classical neural networks on a quantum computer. They use the principles of quantum mechanics to perform computations that are difficult or impossible to perform on classical computers.

The basic idea of a quantum neural network is to encode the input data into a quantum state, and then apply a sequence of quantum gates to transform the state into an output state that represents the output of the neural network.

<h3 style="color:#D10283;">QNN Approaches</h3>

- The quantum circuit approach uses a parameterized quantum circuit to perform the computations, where the parameters are trained using an optimization algorithm to minimize a loss function.

- The quantum tensor network approach uses tensor network methods to represent the quantum states and perform the computations.

<h3 style="color:#D10283;">QNN Applications</h3>


- quantum machine learning
- quantum data analysis
- quantum control 
- to accelerate classical neural network computations by using a quantum computer as a co-processor for certain tasks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import qiskit
from qiskit import transpile, assemble
from qiskit.visualization import *

class QuantumCircuit:
    """ 
    This class provides a simple interface for interaction 
    with the quantum circuit 
    """
    
    def __init__(self, n_qubits, backend, shots):
        # --- Circuit definition ---
        self._circuit = qiskit.QuantumCircuit(n_qubits)
        
        all_qubits = [i for i in range(n_qubits)]
        self.theta = qiskit.circuit.Parameter('theta')
        
        self._circuit.h(all_qubits)
        self._circuit.barrier()
        self._circuit.ry(self.theta, all_qubits)
        
        self._circuit.measure_all()
        # ---------------------------

        self.backend = backend
        self.shots = shots
    
    def run(self, thetas):
        t_qc = transpile(self._circuit,
                         self.backend)
        qobj = assemble(t_qc,
                        shots=self.shots,
                        parameter_binds = [{self.theta: theta} for theta in thetas])
        job = self.backend.run(qobj)
        result = job.result().get_counts()
        
        counts = np.array(list(result.values()))
        states = np.array(list(result.keys())).astype(float)
        
        # Compute probabilities for each state
        probabilities = counts / self.shots
        # Get state expectation
        expectation = np.sum(states * probabilities)
        
        return np.array([expectation])
    
    
class HybridFunction(Function):
    """ Hybrid quantum - classical function definition """
    
    @staticmethod
    def forward(ctx, input, quantum_circuit, shift):
        """ Forward pass computation """
        ctx.shift = shift
        ctx.quantum_circuit = quantum_circuit

        expectation_z = ctx.quantum_circuit.run(input[0].tolist())
        result = torch.tensor([expectation_z])
        ctx.save_for_backward(input, result)

        return result
        
    @staticmethod
    def backward(ctx, grad_output):
        """ Backward pass computation """
        input, expectation_z = ctx.saved_tensors
        input_list = np.array(input.tolist())
        
        shift_right = input_list + np.ones(input_list.shape) * ctx.shift
        shift_left = input_list - np.ones(input_list.shape) * ctx.shift
        
        gradients = []
        for i in range(len(input_list)):
            expectation_right = ctx.quantum_circuit.run(shift_right[i])
            expectation_left  = ctx.quantum_circuit.run(shift_left[i])
            
            gradient = torch.tensor([expectation_right]) - torch.tensor([expectation_left])
            gradients.append(gradient)
        gradients = np.array([gradients]).T
        return torch.tensor([gradients]).float() * grad_output.float(), None, None

class Hybrid(nn.Module):
    """ Hybrid quantum - classical layer definition """
    
    def __init__(self, backend, shots, shift):
        super(Hybrid, self).__init__()
        self.quantum_circuit = QuantumCircuit(1, backend, shots)
        self.shift = shift
        
    def forward(self, input):
        return HybridFunction.apply(input, self.quantum_circuit, self.shift)

<h1 style="color:#510981;">VQLS: The Variational Quantum Linear Solver</h1>

The Variational Quantum Linear Solver (VQLS) is a quantum algorithm that can be used to solve linear systems of equations. It is a hybrid algorithm that combines classical and quantum computations to solve the problem.

Specifically, if we are given some matrix  
A , such that A|x⟩ = |b⟩ , where |b⟩  is some known vector, the VQLS algorithm is theoretically able to find a normalized |x⟩  that is proportional to |x⟩ , which makes the above relationship true.

<h3 style="color:#D10283;">The VQLS algorithm involves two main steps</h3>


- preparing a quantum state that encodes the solution to the linear system
- measuring the quantum state to obtain the solution to the linear system.

Firstly, the quantum state is prepared using a parameterized quantum circuit that can be optimized to encode the solution to the linear system. The circuit is designed to implement a unitary transformation that maps an input state to the solution state. 

Then, the quantum state is measured to obtain the solution to the linear system. The measurement is performed in the computational basis, which means that the quantum state collapses to a classical state that represents the solution. 

In [ ]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, transpile, assemble
import math
import random
import numpy as np
from scipy.optimize import minimize

def apply_fixed_ansatz(qubits, parameters):

    for iz in range (0, len(qubits)):
        circ.ry(parameters[0][iz], qubits[iz])

    circ.cz(qubits[0], qubits[1])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[1][iz], qubits[iz])

    circ.cz(qubits[1], qubits[2])
    circ.cz(qubits[2], qubits[0])

    for iz in range (0, len(qubits)):
        circ.ry(parameters[2][iz], qubits[iz])

circ = QuantumCircuit(3)
apply_fixed_ansatz([0, 1, 2], [[1, 1, 1], [1, 1, 1], [1, 1, 1]])
circ.draw()

# Creates the Hadamard test

def had_test(gate_type, qubits, auxiliary_index, parameters):

    circ.h(auxiliary_index)

    apply_fixed_ansatz(qubits, parameters)

    for ie in range (0, len(gate_type[0])):
        if (gate_type[0][ie] == 1):
            circ.cz(auxiliary_index, qubits[ie])

    for ie in range (0, len(gate_type[1])):
        if (gate_type[1][ie] == 1):
            circ.cz(auxiliary_index, qubits[ie])
    
    circ.h(auxiliary_index)
    
circ = QuantumCircuit(4)
had_test([[0, 0, 0], [0, 0, 1]], [1, 2, 3], 0, [[1, 1, 1], [1, 1, 1], [1, 1, 1]])
circ.draw()

# Creates controlled anstaz for calculating |<b|psi>|^2 with a Hadamard test

def control_fixed_ansatz(qubits, parameters, auxiliary, reg):

    for i in range (0, len(qubits)):
        circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[1], 4)
    circ.cz(qubits[0], 4)
    circ.ccx(auxiliary, qubits[1], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[1][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    circ.ccx(auxiliary, qubits[2], 4)
    circ.cz(qubits[1], 4)
    circ.ccx(auxiliary, qubits[2], 4)

    circ.ccx(auxiliary, qubits[0], 4)
    circ.cz(qubits[2], 4)
    circ.ccx(auxiliary, qubits[0], 4)

    for i in range (0, len(qubits)):
        circ.cry(parameters[2][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

q_reg = QuantumRegister(5)
circ = QuantumCircuit(q_reg)
control_fixed_ansatz([1, 2, 3], [[1, 1, 1], [1, 1, 1], [1, 1, 1]], 0, q_reg)
circ.draw()